# Data Quality Monitoring with DMFs

This template demonstrates how to implement comprehensive data quality monitoring using Snowflake's Data Metric Functions (DMFs). You'll learn to create custom quality metrics, set up automated monitoring dashboards, and establish alerting mechanisms to proactively identify data issues before they impact your analytics and business decisions.

**What you'll accomplish:**
- Create sample data with intentional quality issues for demonstration
- Build custom Data Metric Functions (DMFs) to measure completeness, uniqueness, validity, and consistency  
- Set up a monitoring dashboard to track quality metrics over time
- Implement threshold-based alerting to detect quality degradation

**Time to complete:** ~5 minutes

## Step 1: Environment Setup

Set up our Snowflake environment and prepare for data quality monitoring.

In [ ]:
-- Set up the learning environment
USE ROLE SNOWFLAKE_LEARNING_ROLE;
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;
USE DATABASE SNOWFLAKE_LEARNING_DB;
SET schema_name = CONCAT(CURRENT_USER(), '_DATA_QUALITY_MONITOR');
USE SCHEMA IDENTIFIER($schema_name);

-- Clean up any existing objects from previous runs
DROP TABLE IF EXISTS CUSTOMER_TRANSACTIONS;
DROP FUNCTION IF EXISTS completeness_check;
DROP FUNCTION IF EXISTS uniqueness_check;
DROP FUNCTION IF EXISTS validity_check;
DROP FUNCTION IF EXISTS outlier_check;
DROP VIEW IF EXISTS DATA_QUALITY_DASHBOARD;

## Step 2: Create Sample Data

We'll create a sample dataset with intentional quality issues:
- **Missing values** (null amounts and categories)
- **Duplicate records** (identical transactions)
- **Invalid formats** (malformed emails)
- **Outliers** (unusually high amounts)

In [ ]:
-- Create the sample transactions table
CREATE OR REPLACE TABLE CUSTOMER_TRANSACTIONS (
    CUSTOMER_ID INT,
    TRANSACTION_DATE DATE,
    AMOUNT DECIMAL(10,2),
    CATEGORY VARCHAR(50),
    EMAIL VARCHAR(100),
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

In [ ]:
-- Insert sample data with quality issues
INSERT INTO CUSTOMER_TRANSACTIONS (CUSTOMER_ID, TRANSACTION_DATE, AMOUNT, CATEGORY, EMAIL) VALUES
(1001, '2024-01-15', 45.67, 'GROCERY', 'customer1001@email.com'),
(1002, '2024-01-16', NULL, 'RETAIL', 'customer1002@email.com'), -- Missing amount
(1003, '2024-01-17', 123.45, NULL, 'customer1003@email.com'), -- Missing category
(1001, '2024-01-15', 45.67, 'GROCERY', 'customer1001@email.com'), -- Duplicate
(1004, '2024-01-18', 89.99, 'RESTAURANT', 'invalid_email_1004'), -- Invalid email
(1005, '2024-01-19', 8750.00, 'ONLINE', 'customer1005@email.com'), -- Outlier amount
(1006, '2024-01-20', 67.34, 'GAS', 'customer1006@email.com'),
(1007, '2024-01-21', 234.56, 'RETAIL', 'customer1007@email.com'),
(1008, '2024-01-22', NULL, 'GROCERY', 'customer1008@email.com'), -- Missing amount
(1009, '2024-01-23', 156.78, 'RESTAURANT', 'bad-email-format'), -- Invalid email
(1010, '2024-01-24', 78.90, NULL, 'customer1010@email.com'), -- Missing category
(1006, '2024-01-20', 67.34, 'GAS', 'customer1006@email.com'), -- Duplicate
(1011, '2024-01-25', 9250.50, 'ONLINE', 'customer1011@email.com'); -- Outlier amount

-- Add more realistic data to reach meaningful sample size
INSERT INTO CUSTOMER_TRANSACTIONS (CUSTOMER_ID, TRANSACTION_DATE, AMOUNT, CATEGORY, EMAIL)
SELECT 
    UNIFORM(2000, 9999, RANDOM()) as customer_id,
    DATEADD('day', -UNIFORM(1, 365, RANDOM()), CURRENT_DATE()) as transaction_date,
    ROUND(ABS(NORMAL(100, 50, RANDOM())), 2) as amount,
    CASE UNIFORM(1, 5, RANDOM()) 
        WHEN 1 THEN 'GROCERY'
        WHEN 2 THEN 'RETAIL'
        WHEN 3 THEN 'RESTAURANT'
        WHEN 4 THEN 'GAS'
        ELSE 'ONLINE'
    END as category,
    'customer' || UNIFORM(2000, 9999, RANDOM()) || '@email.com' as email
FROM TABLE(GENERATOR(ROWCOUNT => 500));

In [ ]:
-- View sample of the data and basic stats
SELECT COUNT(*) as total_records FROM CUSTOMER_TRANSACTIONS;

SELECT * FROM CUSTOMER_TRANSACTIONS LIMIT 10;

## Step 3: Basic Quality Assessment

Let's examine the quality issues in our dataset manually first.

In [ ]:
-- Basic quality assessment
SELECT 
    COUNT(*) as total_records,
    COUNT_IF(AMOUNT IS NULL) as null_amounts,
    ROUND(COUNT_IF(AMOUNT IS NULL) / COUNT(*) * 100, 2) as null_amounts_pct,
    COUNT_IF(CATEGORY IS NULL) as null_categories,
    ROUND(COUNT_IF(CATEGORY IS NULL) / COUNT(*) * 100, 2) as null_categories_pct,
    COUNT(*) - COUNT(DISTINCT CUSTOMER_ID, TRANSACTION_DATE, AMOUNT, CATEGORY) as duplicate_records,
    COUNT_IF(NOT EMAIL REGEXP '^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$') as invalid_emails
FROM CUSTOMER_TRANSACTIONS;

## Step 4: Create Data Metric Functions (DMFs)

Data Metric Functions allow us to create reusable, standardized quality checks. We'll create four DMFs to monitor different aspects of data quality:

1. **Completeness Check** - Measures percentage of non-null values
2. **Uniqueness Check** - Identifies duplicate records  
3. **Validity Check** - Validates email format correctness
4. **Outlier Check** - Detects values outside normal ranges

In [ ]:
-- 1. Completeness Check DMF
CREATE OR REPLACE DATA METRIC FUNCTION completeness_check(
    ARG_T TABLE(AMOUNT NUMBER(10,2))
)
RETURNS NUMBER(10,2)
LANGUAGE SQL
AS
$$
    SELECT 
        (COUNT(*) - COUNT_IF(AMOUNT IS NULL)) / COUNT(*) * 100 AS completeness_percentage
    FROM ARG_T
$$;

In [ ]:
-- 2. Uniqueness Check DMF
CREATE OR REPLACE DATA METRIC FUNCTION uniqueness_check(
    ARG_T TABLE(CUSTOMER_ID INT, TRANSACTION_DATE DATE, AMOUNT NUMBER(10,2), CATEGORY VARCHAR(50))
)
RETURNS NUMBER(10,2)
LANGUAGE SQL  
AS
$$
    SELECT 
        COUNT(DISTINCT CUSTOMER_ID, TRANSACTION_DATE, AMOUNT, CATEGORY) / COUNT(*) * 100 AS uniqueness_percentage
    FROM ARG_T
$$;

In [ ]:
-- 3. Email Validity Check DMF
CREATE OR REPLACE DATA METRIC FUNCTION validity_check(
    ARG_T TABLE(EMAIL VARCHAR(100))
)
RETURNS NUMBER(10,2)
LANGUAGE SQL
AS
$$
    SELECT 
        COUNT_IF(EMAIL REGEXP '^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$') / COUNT(*) * 100 AS validity_percentage
    FROM ARG_T
$$;

In [ ]:
-- 4. Outlier Detection DMF (using 3-sigma rule)
CREATE OR REPLACE DATA METRIC FUNCTION outlier_check(
    ARG_T TABLE(AMOUNT NUMBER(10,2))
)
RETURNS NUMBER(10,2)
LANGUAGE SQL
AS
$$
    WITH stats AS (
        SELECT 
            AVG(AMOUNT) AS mean_amount,
            STDDEV(AMOUNT) AS stddev_amount
        FROM ARG_T
        WHERE AMOUNT IS NOT NULL
    )
    SELECT 
        COUNT_IF(ABS(AMOUNT - mean_amount) <= 3 * stddev_amount) / COUNT(*) * 100 AS non_outlier_percentage
    FROM ARG_T, stats
    WHERE AMOUNT IS NOT NULL
$$;

## Step 5: Test Data Metric Functions

Now let's test our DMFs to see how they measure the quality of our sample data.

In [ ]:
-- Test all DMFs on our sample data
SELECT 
    'Data Quality Metrics' as metric_type,
    completeness_check(SELECT AMOUNT FROM CUSTOMER_TRANSACTIONS) AS amount_completeness_pct,
    uniqueness_check(SELECT CUSTOMER_ID, TRANSACTION_DATE, AMOUNT, CATEGORY FROM CUSTOMER_TRANSACTIONS) AS record_uniqueness_pct,
    validity_check(SELECT EMAIL FROM CUSTOMER_TRANSACTIONS) AS email_validity_pct,
    outlier_check(SELECT AMOUNT FROM CUSTOMER_TRANSACTIONS) AS non_outlier_pct;

## Step 6: Create Monitoring Dashboard

Create a view that serves as our data quality monitoring dashboard. This view can be scheduled to run regularly for continuous monitoring.

In [ ]:
-- Create data quality monitoring dashboard
CREATE OR REPLACE VIEW DATA_QUALITY_DASHBOARD AS
SELECT 
    'CUSTOMER_TRANSACTIONS' AS table_name,
    CURRENT_TIMESTAMP() AS check_time,
    completeness_check(SELECT AMOUNT FROM CUSTOMER_TRANSACTIONS) AS amount_completeness,
    uniqueness_check(SELECT CUSTOMER_ID, TRANSACTION_DATE, AMOUNT, CATEGORY FROM CUSTOMER_TRANSACTIONS) AS record_uniqueness,
    validity_check(SELECT EMAIL FROM CUSTOMER_TRANSACTIONS) AS email_validity,
    outlier_check(SELECT AMOUNT FROM CUSTOMER_TRANSACTIONS) AS data_consistency;

-- View the dashboard
SELECT * FROM DATA_QUALITY_DASHBOARD;

## Step 7: Implement Quality Alerting

Set up threshold-based alerting to detect when data quality falls below acceptable levels.

In [ ]:
-- Quality alerting with thresholds
WITH quality_thresholds AS (
    SELECT 
        95.0 as completeness_threshold,
        97.0 as uniqueness_threshold,
        98.0 as validity_threshold,
        99.0 as consistency_threshold
),
current_quality AS (
    SELECT * FROM DATA_QUALITY_DASHBOARD
)
SELECT 
    table_name,
    check_time,
    CASE 
        WHEN amount_completeness < completeness_threshold THEN 'ALERT: Amount completeness below ' || completeness_threshold || '%'
        ELSE 'OK'
    END as completeness_status,
    CASE 
        WHEN record_uniqueness < uniqueness_threshold THEN 'ALERT: Record uniqueness below ' || uniqueness_threshold || '%'
        ELSE 'OK'
    END as uniqueness_status,
    CASE 
        WHEN email_validity < validity_threshold THEN 'ALERT: Email validity below ' || validity_threshold || '%'
        ELSE 'OK'
    END as validity_status,
    CASE 
        WHEN data_consistency < consistency_threshold THEN 'ALERT: Data consistency below ' || consistency_threshold || '%'
        ELSE 'OK'
    END as consistency_status
FROM current_quality, quality_thresholds;

## Key Takeaways

🎯 **What you've learned:**

1. **Proactive Quality Monitoring**: Data Metric Functions enable automated, continuous monitoring of data quality across your pipelines

2. **Custom Quality Metrics**: You can create reusable DMFs tailored to your specific data quality requirements (completeness, uniqueness, validity, consistency)

3. **Threshold-Based Alerting**: Quality thresholds help you catch issues early before they impact downstream analytics and business decisions

4. **Scalable Monitoring**: The dashboard approach scales across multiple tables and can be scheduled for regular execution

**Next steps for production use:**
- Schedule the dashboard view to run regularly using Snowflake Tasks
- Integrate alerts with your notification system (email, Slack, etc.)
- Expand DMFs to cover additional quality dimensions specific to your data
- Set up historical tracking to monitor quality trends over time

## Cleanup

Clean up the resources created in this template.

In [ ]:
-- Clean up created objects
DROP VIEW IF EXISTS DATA_QUALITY_DASHBOARD;
DROP TABLE IF EXISTS CUSTOMER_TRANSACTIONS;
DROP FUNCTION IF EXISTS completeness_check;
DROP FUNCTION IF EXISTS uniqueness_check;
DROP FUNCTION IF EXISTS validity_check;
DROP FUNCTION IF EXISTS outlier_check;

SELECT 'Cleanup completed successfully!' as status;

## Additional Resources

📚 **Learn more about data quality in Snowflake:**

- [Snowflake Data Quality Documentation](https://docs.snowflake.com/en/user-guide/data-quality-intro)
- [Data Metric Functions Reference](https://docs.snowflake.com/en/sql-reference/data-metric-functions)
- [Data Quality Best Practices](https://docs.snowflake.com/en/user-guide/data-quality-best-practices)

🔗 **Explore more templates:**
- Visit the [Snowflake Templates Hub](https://app.snowflake.com/templates) for additional examples
- Try related templates on data governance and monitoring

---
*This template demonstrates Snowflake's data quality monitoring capabilities using Data Metric Functions for proactive data health management.*